In [19]:
import pydot
# path = "/home/fzc/workspace/convul/cpg_single/cpg_process/cpg.dot"
# data = path.read()
pydot.graph_from_dot_data("/home/fzc/workspace/convul/cpg_single/cpg_process/cpg.dot")

/home/fzc/workspace/convul/cpg_single/cpg_process/cpg.dot
^
Expected {'GRAPH' | 'DIGRAPH'}, found '/'  (at char 0), (line:1, col:1)


In [2]:
import pydot

# 读取 .dot 文件c
(graph,) = pydot.graph_from_dot_file("/home/fzc/workspace/convul/cpg_single/cpg_process/cpg.dot")
pdg = pydot.graph_from_dot_file("/home/fzc/workspace/convul/cpg_single/cpg_process/cpg.dot")
print('pdg:',pdg)
# pydot.graph_from_dot_data(graph)
# 输出图的基本信息
print("Nodes:", graph.get_nodes())
print("Edges:", graph.get_edges())
# print(graph)
# 输出每个节点的信息
# print(graph.nodes())
print("Nodes:")
for node in graph.get_nodes():
    print(f"Node ID: {node.get_name()}, Label: {node.get_label()}")
    

# 输出每条边的信息
print("\nEdges:")
for edge in graph.get_edges():
    print(f"Edge from {edge.get_source()} to {edge.get_destination()}")



pdg: [<pydot.core.Dot object at 0x7c4223891c40>]
Nodes: [<pydot.core.Node object at 0x7c424621b9a0>, <pydot.core.Node object at 0x7c422336b790>, <pydot.core.Node object at 0x7c422336b7f0>, <pydot.core.Node object at 0x7c422336b730>, <pydot.core.Node object at 0x7c422336bc70>, <pydot.core.Node object at 0x7c422336b4f0>, <pydot.core.Node object at 0x7c422336b9d0>, <pydot.core.Node object at 0x7c422336b700>, <pydot.core.Node object at 0x7c422336b340>, <pydot.core.Node object at 0x7c422336b7c0>, <pydot.core.Node object at 0x7c422336b490>, <pydot.core.Node object at 0x7c422336b850>, <pydot.core.Node object at 0x7c422336bdf0>, <pydot.core.Node object at 0x7c422336b310>, <pydot.core.Node object at 0x7c422336ba90>, <pydot.core.Node object at 0x7c422336bd00>, <pydot.core.Node object at 0x7c422336b5e0>, <pydot.core.Node object at 0x7c422336bf40>, <pydot.core.Node object at 0x7c422336b820>]
Edges: [<pydot.core.Edge object at 0x7c422336b2e0>, <pydot.core.Edge object at 0x7c422336baf0>, <pydot.core

In [17]:
import pydot

dot_string = """graph my_graph {
    bgcolor="yellow";
    a [label="Foo"];
    b [shape=circle];
    a -- b -- c [color=blue];
}"""

graphs = pydot.graph_from_dot_data(dot_string)
graph = graphs[0]
print(graph)

graph my_graph {
bgcolor="yellow";
a [label="Foo"];
b [shape=circle];
a -- b [color=blue];
b -- c [color=blue];
}



In [ ]:
# 把目录下的所有文件分为train和test 分别写入nvd_train.txt和nvd_test.txt,每一行是一个文件的路径
import os
import random
base_dir = './deeprace_preprocess/nvd/'
def split_files_to_train_test(directory, train_ratio=0.8):
    # 获取文件目录下的所有文件路径
    files = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    
    # 打乱文件顺序
    random.shuffle(files)
    
    # 按照train_ratio分割成train和test
    train_count = int(len(files) * train_ratio)
    train_files = files[:train_count]
    test_files = files[train_count:]
    
    # 写入nvd_train.txt
    with open(base_dir+"nvd_train.txt", "w") as train_file:
        for file_path in train_files:
            train_file.write(f"{file_path}\n")
    
    # 写入nvd_test.txt
    with open(base_dir+"nvd_test.txt", "w") as test_file:
        for file_path in test_files:
            test_file.write(f"{file_path}\n")

# 使用目录路径和训练集比例调用函数
# split_files_to_train_test("/home/fzc/dataset/all_data_preprocess/final/dataset/", train_ratio=0.8)
split_files_to_train_test("/home/fzc/dataset/cope_deeprace/3_out_json_wi/", train_ratio=0.8)


In [6]:
import torch
from torch import nn
from torch_geometric.nn import RGCNConv, global_add_pool

class RGCN(nn.Module):
    def __init__(self, output_dim=200, input_dim=100, num_relations=5):
        super().__init__()
        self.conv = RGCNConv(input_dim, output_dim, num_relations)
        hidden_size = 256
        layers = [
            nn.Linear(output_dim, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.3)
        ]
        layers += [
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.3)
        ]
        self.__hidden_layers = nn.Sequential(*layers)
        self.__classifier = nn.Linear(hidden_size, 2)
        self.readout = global_add_pool
        self.softmax = nn.Sigmoid()

    def forward(self, x, edge_index, edge_type):
        x = x.to(torch.device("cuda:0"))
        edge_index = edge_index.to(torch.device("cuda:0"))
        edge_type = edge_type.to(torch.device("cuda:0"))
        outputs = self.conv(x, edge_index, edge_type)
        pooled = self.readout(outputs, torch.zeros(outputs.shape[0], dtype=int, device=outputs.device))
        hiddens = self.__hidden_layers(pooled)
        avg = self.__classifier(hiddens)
        result = self.softmax(avg)
        return result

# 示例数据
x = torch.randn((10, 100))  # 10个节点，每个节点有100维特征
x = x.to(torch.device("cuda:0"))
edge_index = torch.tensor([[0, 1, 2, 3], [1, 2, 3, 4]], dtype=torch.long)  # 边索引
edge_index = edge_index.to(torch.device("cuda:0"))
edge_type = torch.tensor([0, 1, 2, 0], dtype=torch.long)  # 边类型
print(edge_type.shape)
edge_type = edge_type.to(torch.device("cuda:0"))

# 创建模型并进行前向传播
model = RGCN(output_dim=200, input_dim=100, num_relations=3)
model = model.to(torch.device("cuda:0"))
output = model(x, edge_index, edge_type)
print(output)

torch.Size([4])
tensor([[0.3053, 0.5402]], device='cuda:0', grad_fn=<SigmoidBackward0>)
